# Neuropsychology: Working with Scientific Data

This lesson is based on:
* https://github.com/btel/python-in-neuroscience-tutorials
* https://github.com/voytekresearch/tutorials

![](figs/L10/sphx_glr_mri_with_eeg_001.png)
Made with Python: https://matplotlib.org/gallery/specialty_plots/mri_with_eeg.html#sphx-glr-gallery-specialty-plots-mri-with-eeg-py

# What does EEG data usually look like?

EEG data is usually a matrix, which is a [table of numbers](https://www.mathsisfun.com/algebra/matrix-introduction.html). 
For example, this is a very small matrix (called an array in Python):

$\begin{bmatrix}1 & 2 & 3 \\4 & 5 & 6 \\ 7 & 8 & 9\end{bmatrix}$


# How do we work with matrices in Python? [Numpy](https://www.numpy.org/devdocs/user/quickstart.html)

In [ ]:
import matplotlib.cbook as cbook # matplotlib - data visualization library
import numpy as np # numpy - matrix math library

In [Lecture 05: Files & Strings](L05_files_strings.ipynb), we manually parsed (tore apart) the file to get out the data. In this case, because the eeg data is a .dat file, we can use the [numpy](https://www.numpy.org/devdocs/user/quickstart.html) to open and parse it. Here we load the data from the plot seen above. 

We know before hand that we have 4 electrods [PG3, PG5 PG7, PG9] and 800 samples per electrodes. We use this information to put the numbers in the file into the table correctly.

In [ ]:
# Load the EEG data (this is the sample data used in the plot above)
n_samples, n_rows = 800, 4
with cbook.get_sample_data('eeg.dat') as eegfile:
    # fromfile gets a list of numbers, reshape puts it into [800 by 4 form]
    data = np.fromfile(eegfile, dtype=float).reshape((n_samples, n_rows))

In [ ]:
# How big is data? Use the .shape attribute of a numpy array
data.shape

In [ ]:
# What does the data look like? 
data

# How do we get measurements out? Indexing

Like in [lecture 06: Data Exploration](L06_stats.ipynb), numpy uses the row/column convention.
 ![image of axis, where rows=axis 0, columns = axis 1](figs/L06/axis.jpg)
[stackoverflow](https://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition)

[Indexing on an array](http://www.pythoninformer.com/python-libraries/numpy/index-and-slice/) is similar to indexing on [lists](L04_lists_dicts.ipynb). Given:

In [ ]:
a = np.arange(1,10).reshape(3,3)
a

# Selecting a row:

In [ ]:
a[1]


# Selecting a column:
The conventions is A[row, column] and : means return all the elements of that dimension. 

In [ ]:
a[:,1]

# Selecting both:
Rows and columns can be subset at the same time:

In [ ]:
a[:2,:2]

In [ ]:
# Also valid
a[0:2, 0:2]

# Pair up & practice: 
1. Extract the samples for the 2nd electrode in the data
2. Extract the 500th sample for all the electrodes
3. Extract the first 100 samples of the 2nd and 3rd electrodes


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
_ = ax.plot(data)

How can we make them not crowded? Let's 
1. Associate each row with it's electrode
2. Create an axes for electrode 
3. Make the figures [share axis](L07_stats_viz.ipynb) so that the Xs and Ys line up


In [ ]:
# use zip to pair electrodes with labels:
list(zip(['PG3', 'PG5', 'PG7', 'PG9'], data))

# Is that correct? 

zip is pairing up the labels with the rows, which here are the samples 
flip rows and columns using .T - transpose

# What is transpose?

In [ ]:
a.T

In [ ]:
list(zip(['PG3', 'PG5', 'PG7', 'PG9'], data.T))

In [ ]:
fig, axes = plt.subplots(nrows=4, sharex=True, sharey = True, figsize=(15,5))
for ax, label, electrode in zip(axes, ['PG3', 'PG5', 'PG7', 'PG9'], data.T):
    _ = ax.plot(electrode)
    _ = ax.set_ylabel(label)
ax.set_xlim(0,800) # set the starting and ending points of the graph
plt.subplots_adjust(hspace=0)# no space between graphs

# That's some spikey data! Can we process it for less spikiness?

Let's convert our data into a Pandas data frame because Pandas provides a lot of functions natively:


In [ ]:
import pandas as pd
df = pd.DataFrame(dict(zip(['PG3', 'PG5', 'PG7', 'PG9'], data.T)))

In [ ]:
df.head()

In [ ]:
# Let's check that it looks the same as above
fig, ax = plt.subplots()
_ = df.plot(ax=ax)
_ = ax.legend(ncol=4) # modify the legend

Lots of tangle! How do we get out 'PG3'?

In [ ]:
df['PG3'].head()

In [ ]:
_ = df['PG3'].plot()

# How do we smooth this signal? - rolling average!

The EEG data is defined such that for every time (t) there's a measurement of the voltage of the electrical signal passing through the brain.

In [ ]:
fig, ax = plt.subplots()
_ = df['PG3'].plot(ax=ax)
_ = ax.set_xlabel("time")
_ = ax.set_ylabel("EEG signal (Volts)")

To compute the rolling average:
1. Define a `window_size`, which is the number of observations to include in the average.
2. Set time t=0. Define a window between t=0 and t=window_size (red box in the figure below)
3. Take the average of the voltages in that window and store that as moving_average(t) (black line)
4. Shift t and the moving average by 1, such that the window is between t=1, and t=1+window_size
5. Take the average of the volatages in the new window, and store as moving_average(t+1)
6. Repeat for all values in (t)

In [ ]:
N=100
dfsub = df['PG3'][:N] # taking a subset so the animation doesn't explode
window_size = 10
moving_avg = dfsub.rolling(window_size).mean()

In [ ]:
%%capture
import matplotlib.patches as mpatches
from matplotlib.widgets import Slider, Button, RadioButtons


fig, ax = plt.subplots(figsize=(15,5))
dfsub.plot(ax=ax, label='PG3')
line, = ax.plot(0, dfsub[0], linewidth=3, color='k', label="moving average")
ax.set_xticks(np.arange(0,N, window_size))
ax.xaxis.grid(True)
ax.legend(ncol=2)
def animate(i):
    mwindow = ax.axvspan(0, i+window_size, color='white')
   
    mwindow = ax.axvspan(i, i+window_size, color='red', alpha=.25)
    
    line.set_data(dfsub.index[:i+1], moving_avg[:i+1])
    return [mwindow, line]

In [ ]:
from matplotlib import animation
from IPython.display import HTML

ani = animation.FuncAnimation(fig, animate, frames=N)

In [ ]:
HTML(ani.to_jshtml())

# Practice:
What happens when you change the step? How does the moving average change?

# Can we apply this to all the data?

In [ ]:
df.head()

In [ ]:
mv = df.rolling(window_size).mean()

In [ ]:
_ = mv.plot()

# Pair up & Practice:
1. Separate out the original signals into their own axes
2. Seperate out the moving averages into their own axes
3. Try different window sizes and see how things change